### Importa o Gurobi

In [19]:
from gurobipy import Model, GRB, quicksum

### Recebe os dados do arquivo

In [20]:
from ler_arquivo import ler_arquivo

n_arquivo = input("Escolha um número de 1 a 10")
delta, T, m, recursos, lista_adjacencia, vertices = ler_arquivo(f"instances/fn{n_arquivo}.dat")

### Declara o modelo

In [21]:
modelo = Model("fake_news")

### Variáveis de decisão do modelo


$ ∀ s \in V \text{ e } ∀ (u,v) \in E$: \
$e_{u,v,s} \in \{0, 1\}$ =
$\begin{cases}
1, & \text{se o arco (u,v) é utilizado considerando o vértice inicial de propagação s } \\ 
0, & \text{caso contrário} 
\end{cases}$ 

$ ∀ i,s \in V$: \
$t_{i,s} \in \Z$ = tempo de chegada da fake news no vértice i considerando o vértice inicial de propagação s 

$ ∀ i,s \in V$: \
$y_{i, s} \in \{0, 1\}$ =
$\begin{cases}
1, & \text{se a fake news chega no vértice i ao sair de s antes do tempo T } \\
0, & \text{caso contrário} 
\end{cases}$ 

$N \in \Z$ = número mínimo de vértices aos quais a fake news chegou antes de T, considerando todos os vértices como iniciais 

$ ∀ i \in V \text{ e } ∀ j \in \beta$: \
$r_{i, j} \in \{0, 1\}$ = 
$\begin{cases}
1, & \text{se o vértice i possui um recurso que se inicia no tempo j } \\ 
0, & \text{caso contrário}
\end{cases}$ 

$ ∀ i \in V \text{ e } ∀ j \in \beta$: \
$a_{i, j} \in \{0, 1\}$ = 
$\begin{cases}
1, & \text{se o recurso j está efetivamente ativado no vértice i } \\ 
0, & \text{caso contrário}
\end{cases}$


### Declara as variáveis de decisão em código Python

In [22]:
x = {}
for v in vertices:
    x[v] = modelo.addVar(vtype=GRB.BINARY, name=f'{v}')

t = {}
y = {}
for v in vertices:
    for s in vertices:
        t[(v, s)] = modelo.addVar(vtype=GRB.INTEGER, name=f'{v}-{s}')
        y[(v, s)] = modelo.addVar(vtype=GRB.BINARY, name=f'{v}-{s}')

e = {}
for s in vertices:
    for u, arco in lista_adjacencia.items():
        for v, _ in arco:
            e[(u, v, s)] = modelo.addVar(vtype=GRB.BINARY, name=f'{u}-{v}-{s}')

tempos = recursos.keys()
r = {}
a = {}
for v in vertices:
    for tempo in tempos:
        r[(v, tempo)] = modelo.addVar(vtype=GRB.BINARY, name=f'{v}-{tempo}')
        a[(v, tempo)] = modelo.addVar(vtype=GRB.BINARY, name=f'{v}-{tempo}')

N = modelo.addVar(vtype=GRB.INTEGER, name="N")

## Restrições:

* N deve ser um limitante superior para o número de servidores infectados pela *fake news*, considerando todos os servidores como possíveis propagadores iniciais da *fake news*


$$
\sum_{v \in V}y_{i, s} \le N \qquad ∀s \in V
$$

In [23]:
for s in vertices:
    somatorio = quicksum(y[(vertice, s)] for vertice in vertices)
    modelo.addConstr(somatorio <= N)

* O servidor inicial está necessariamente infectado

$$
y_{s, s} = 1 \qquad ∀s \in V
$$

In [24]:
for s in vertices:
    modelo.addConstr(y[(s, s)] == 1)

* A quantidade de recursos por servidor não pode ser maior que 1


$$
\sum_{b \in \beta}r_{v, b} \le 1 \qquad ∀v \in V
$$

In [25]:
for v in vertices:
    somatorio = quicksum(r[(v, tempo)] for tempo in tempos)
    modelo.addConstr(somatorio <= 1)

* A quantidade de recursos instalados para cada tempo de ativação não pode ultrapassar um valor $ \alpha $


$$
\sum_{v \in V}r_{v, b} = \alpha_b \qquad ∀b \in \beta
$$

In [26]:
for tempo in tempos:
    somatorio = quicksum(r[(v, tempo)] for v in vertices)
    modelo.addConstr(somatorio == recursos[tempo])

* Caso um servidor não tenha sido infectado em um tempo T, o valor da variável de decisão que contabiliza o tempo de chegada da *fake news* é zerado


$$
t_{v, s} \le T y_{v, s} \qquad ∀v,s \in V 
$$

In [27]:
for v in vertices:
    for s in vertices:
        modelo.addConstr(t[(v, s)] <= T*y[(v, s)])

* Um recurso só pode estar ativado em um servidor caso aquele recurso tenha sido alocado para o mesmo


$$
a_{v, b} \le r_{v, b} \qquad ∀v \in V \qquad ∀b \in \beta  
$$

In [28]:
for v in vertices:
    for tempo in tempos:
        modelo.addConstr(a[(v, tempo)] <= r[(v, tempo)])

* Um enlace só é utilizado para a propagação de *fake news* caso o servidor de origem esteja contaminado com a *fake news*


$$
e_{u, v, s} \le y_{u, s} \qquad ∀s \in V \qquad ∀(u,v) \in E
$$

In [29]:
for s in vertices:
    for u, arco in lista_adjacencia.items():
        for v, _ in arco:
            modelo.addConstr(e[(u, v, s)] <= y[(u, s)])

* Um servidor só é contaminado pela *fake news* caso algum enlace tenha propagado a *fake news* de um outro servidor infectado para este servidor, com exceção do servidor inicial que é quem divulga primeiramente a *fake news*


$$
\sum_{(u,v) \in E}e_{u, v, s} \ge y_{v, s} \qquad ∀v,s \in V \qquad v\neq s  
$$

In [30]:
for s in vertices:
    for v in vertices:
        if(v != s):
            vertices_chegada = []
            for origem, arco in lista_adjacencia.items():
                for destino, _ in arco:
                    if(destino == v):
                        vertices_chegada.append(origem)
            somatorio = quicksum(e[(u, v, s)] for u in vertices_chegada)
            modelo.addConstr(somatorio >= y[(v, s)])

* Se o recurso está ativo no servidor v, então o tempo de chegada contabilizado no servidor v desde o início da propagação da *fake news* deve ser maior ou igual ao tempo de ativação do recurso


$$
t_{v, s} \ge b - M(1 - a_{v, b}) \qquad ∀v,s \in V \qquad ∀b \in \beta  
$$

In [31]:
M = 2000
for s in vertices:
    for v in vertices:
        for tempo in tempos:
            modelo.addConstr(t[(v, s)] >= tempo - M*(1-a[v, tempo]))

* Se o tempo de chegada do contabilizado no servidor v é menor que o tempo de ativação do recurso, então a variável de decisão do recurso é zerada


$$
t_{v, s} \le b - 1 + Ma_{v, b} \qquad ∀v,s \in V \qquad ∀b \in \beta  
$$

In [32]:
for s in vertices:
    for v in vertices:
        for tempo in tempos:
            modelo.addConstr(t[(v, s)] <= tempo - 1 + M*a[v, tempo])

* O tempo de chegada contabilizado no servidor v deve ser maior ou igual ao tempo de chegada contabilizado no servidor u mais o tempo de propagação base do enlace que liga u a v mais o atraso decorrente do recurso, caso ele esteja ativo. Isso só ocorre se o arco que liga u a v estiver ativo.


$$
t_{v, s} \ge t_{u, s} + h_{u, v} + \delta\sum_{b \in \beta}a_{u, b} - M(1- e_{u, v, s}) \qquad ∀s \in V \qquad ∀(u,v) \in E \qquad ∀b \in \beta  
$$

In [33]:
for s in vertices:
    for u, arco in lista_adjacencia.items():
        for v,h in arco:
            somatorio = quicksum(a[(u, tempo)] for tempo in tempos)
            t[(v, s)] >= t[(u, s)] + h + delta*somatorio -M*(1 - e[(u, v, s)])  

### Função objetivo: deve minimizar o N, que representa o limitante superior da quantidade de servidores atingidos pela *fake news* considerando todos os servidores como possíveis propagadores iniciais da *fake_news*

In [34]:
modelo.setObjective(N, GRB.MINIMIZE)

### Otimização do modelo

In [ ]:
modelo.optimize()

### Impressão das soluções:

In [ ]:
if(modelo.status == GRB.OPTIMAL):
  print(f'Valor ótimo: {modelo.objVal}')
  for recurso in r.items():
    for v in vertices:
      for tempo in tempos:
        print(f'{r[v, tempo].x}')